# Intuition of Reinforcement Learning

This notebook is part of Intuition of Reinforcement Learning blog. It has solved shortest path problem of simple graph using reinforcement learning, Q-Learning algorithm. 

### Important imports

In [1]:
!pip install texttable
from texttable import Texttable
import json
from operator import itemgetter

### Initiate state parameters

Function in next cell will be used to initiate state parameters for given state connection

In [2]:
def get_actions_rewards_next_state_q_value_dict(state_connections):
  """
  state_connections: list of tuple, tuple = (next_state, rewards)
  return: dict
  
  e.g. of return dict:
  {
    'actions': [1, 2],
    'value_function': 0,
    'action_for_vf': 0,
    1        : {
                  'next_state': 3,
                  'rewards': -10,
                  'q_value': 0
               }
    2        : {
                  'next_state': 5,
                  'rewards': -15,
                  'q_value': 0
               }
  }
  """
  dic = dict()
  dic['actions'] = [i for i in range(1, len(state_connections)+1)]
  dic['value_function'] = 0
  dic['action_for_vf'] = 1
  for action in dic['actions']:
    dic[action] = dict()
    dic[action]['next_state'] = state_connections[action-1][0]
    dic[action]['rewards'] = state_connections[action-1][1]
    dic[action]['q_value'] = 0
  return dic

### Graph

<img src="./images/shortest_path.PNG">

Function in next cell create above graph, which we have to solve to find shortest path.

In [3]:
def get_state():
  state = dict()
  state['states'] = [1, 2, 3, 4, 5]
  state[1] = get_actions_rewards_next_state_q_value_dict([(5, -18), (3, -9), (2, -7)])
  state[2] = get_actions_rewards_next_state_q_value_dict([(4, -19), (3, -10), (1, -7)])
  state[3] = get_actions_rewards_next_state_q_value_dict([(5, -2), (4, -11), (2, -10), (1, -9)])
  state[4] = get_actions_rewards_next_state_q_value_dict([(6, -6), (3, -11), (2, -19)])
  state[5] = get_actions_rewards_next_state_q_value_dict([(6, -8), (3, -2), (1, -18)])
  state[6] = get_actions_rewards_next_state_q_value_dict([])

  # print(json.dumps(state, sort_keys=False, indent=4))
  return state

### Q-Learning update

Following function will do Q-learning update as follows.


$Q(s, a) = R(s, a) + V(s'), \text{ Q-value for state } s \text{ and action } a, s' \text{ is next state for action } a$

$V(s) = \max_i Q(s, a_i), \text{ Value function of state } s$ 

$a_{v(s)} = \arg(\max_i Q(s, a_i)), \text{ Action for the value function}$

$\text{where,}$

$s \text{ and } s' \in S, \text{ State space, e.g. } S = \{1, 2, 3, 4, 5, 6\}$

$a \in A(s), \text{ Action space in state } s, \text{ e.g. if state is 2 then } A(2) = \{1, 2, 3\}$

$R(s, a) \text{ is rewards for taking an action } a \text{ in state } s, \text{ if state 3 and action is 4 then } R(3, 4) = -9$

$Q(s, a) \text{ is Q-value of state-action pair for state } s \text{ and action } a$ 

In iteration 0, all Q-values are initialize with zero. So value function for each state also will be zero. As all Q-values are same (zero) for all state, so all action value for all state can be initialize with 1 (first action).

Here we will see how Q-value, value function and action for value function is being updated with an example (lets for state 2). Other states will be updated similarly. Assuming we have updated table for iteration 0 and will update for iteration 1. In iteration 1, value function will be used from iteration 0 table.

$Q(2, 1) = R(2, 1) + V(4) = -19 + 0 = -19$

$Q(2, 2) = R(2, 2) + V(3) = -10 + 0 = -10$

$Q(2, 3) = R(2, 3) + V(1) = -7 + 0 = -7$

$V(2) = \max(Q(2, 1), Q(2, 2), Q(2, 3)) = \max(-19, -10, -7) = -7$

$a_{v(2)} = \arg(\max(Q(2, 1), Q(2, 2), Q(2, 3))) = \arg(\max(-19, -10, -7)) = 3$



In [4]:
def update_q_values(state_dic):
  q_dic = dict()
  updated = False
  for state in state_dic['states']:
    for action in state_dic[state]['actions']:
      next_state = state_dic[state][action]['next_state']
      rewards = state_dic[state][action]['rewards']
      next_state_value_fun = state_dic[next_state]['value_function']
      if state_dic[state][action]['q_value'] != rewards + next_state_value_fun:
        state_dic[state][action]['q_value'] = rewards + next_state_value_fun
        updated = True
      
  # update value function
  for state in state_dic['states']:
    q_values = []
    for key in state_dic[state]['actions']:
      q_values.append(state_dic[state][key]['q_value'])
    index, state_dic[state]['value_function'] = max(enumerate(q_values), key=itemgetter(1))
    state_dic[state]['action_for_vf'] = index + 1
    
  return updated
  

### Print Q-table

This function will be used to print Q-table.

In [5]:
def print_q_values_table(state_dic):
    
  max_actions = 0
  for state in state_dic['states']:
    if len(state_dic[state]['actions']) > max_actions:
      max_actions = len(state_dic[state]['actions'])
  print_dic = dict()
  print_dic[0] = ['state/action'] + [i for i in range(1, max_actions+1)] + ['value_function', 'action_for_vf']
  
  for state in state_dic['states']:
    print_dic[state] = [state]
    for action in state_dic[state]['actions']:
      print_dic[state].append(state_dic[state][action]['q_value'])
    for i in range(len(print_dic[state]), max_actions+1):
      print_dic[state].append('')
    print_dic[state].append(state_dic[state]['value_function'])
    print_dic[state].append(state_dic[state]['action_for_vf'])
    
  rows = []
  for key in print_dic.keys():
    rows.append(print_dic[key])
    
  t = Texttable()
  t.add_rows(rows)
  print(t.draw())
  return  

### Final cell

Following cell creating graph, updating Q-values and printing Q-table using above methods.

In [6]:
updated = True
iter_count = 0
state = get_state()
print('\nIteration: {}'.format(iter_count))
print_q_values_table(state)
while updated:
  iter_count += 1
  updated = update_q_values(state)
  if updated:
    print('\nIteration: {}'.format(iter_count))
    print_q_values_table(state)
  


Iteration: 0
+--------------+---+---+---+---+----------------+---------------+
| state/action | 1 | 2 | 3 | 4 | value_function | action_for_vf |
+==============+===+===+===+===+================+===============+
| 1            | 0 | 0 | 0 |   | 0              | 1             |
+--------------+---+---+---+---+----------------+---------------+
| 2            | 0 | 0 | 0 |   | 0              | 1             |
+--------------+---+---+---+---+----------------+---------------+
| 3            | 0 | 0 | 0 | 0 | 0              | 1             |
+--------------+---+---+---+---+----------------+---------------+
| 4            | 0 | 0 | 0 |   | 0              | 1             |
+--------------+---+---+---+---+----------------+---------------+
| 5            | 0 | 0 | 0 |   | 0              | 1             |
+--------------+---+---+---+---+----------------+---------------+

Iteration: 1
+--------------+-----+-----+-----+----+----------------+---------------+
| state/action |  1  |  2  |  3  | 4  | v